In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../../../datasets/Health_insurance.csv')

In [3]:
data.tail()

,age,sex,bmi,children,smoker,region,charges
1333,50,male,30.97,3,no,northwest,10600.5483
1334,18,female,31.92,0,no,northeast,2205.9808
1335,18,female,36.85,0,no,southeast,1629.8335
1336,21,female,25.80,0,no,southwest,2007.9450
1337,61,female,29.07,0,yes,northwest,29141.3603


In [4]:
data['sex_enc'] = data['sex'].map({'female':1,'male':0})
data['smoker_enc'] = data['smoker'].map({'yes':0,'no':1})

In [5]:
X = data[['age','bmi','children','sex_enc','smoker_enc']]
y = data['charges']

In [6]:
from sklearn.model_selection import train_test_split , GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=1)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [9]:
params = {'n_estimators':[10,25,30,100,200,500],'max_depth':[1,2,3,4,5],'min_samples_leaf':[5,10,15,20,25,30]}

In [10]:
Grid_cv = GridSearchCV(RandomForestRegressor(random_state=1),
                       param_grid=params,
                       verbose=3,
                       cv= 5,n_jobs=-1)
Grid_cv.fit(X_train_scaled,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [5, 10, 15, 20, 25, 30],
                         'n_estimators': [10, 25, 30, 100, 200, 500]},
             verbose=3)

In [11]:
Grid_cv.best_params_

{'max_depth': 4, 'min_samples_leaf': 5, 'n_estimators': 25}

In [12]:
Grid_cv.best_score_

np.float64(0.8595358750715553)

In [13]:
models = {'LR':LinearRegression(),
         'KNN3':KNeighborsRegressor(n_neighbors=3),
         'KNN5':KNeighborsRegressor(n_neighbors=5),
         'KNN7':KNeighborsRegressor(n_neighbors=7),
         'SVRp2':SVR(kernel='poly',degree=2),
         'RF':RandomForestRegressor(n_estimators=200,max_depth=4,min_samples_leaf=10,random_state=1)}

In [14]:
for name, model in models.items():
    model.fit(X_train_scaled,y_train)
    score = model.score(X_test_scaled,y_test)
    print(name,score)

LR 0.7314978720846357
KNN3 0.7787926974840552
KNN5 0.7980126045089856
KNN7 0.8006724074791817
SVRp2 -0.05696777857198576
RF 0.8543223439337837


In [15]:
scaler1 = MinMaxScaler()
scaler1.fit(X)
X_scaled = scaler1.transform(X)

In [16]:
final_model = RandomForestRegressor(n_estimators=200,max_depth=4,min_samples_leaf=10,random_state=1)
final_model.fit(X_scaled,y)
final_model.score(X_scaled,y)

0.8735077775171926

In [23]:
#X = data[['age','bmi','children','sex_enc','smoker_enc']]

q = [[22,23.4,0,1,1]]
Q_scaled = scaler.transform(q)

C:\Users\91965\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [24]:
final_model.predict(Q_scaled)

array([3019.59004835])

In [19]:
import pickle 

In [20]:
pickle.dump(final_model,open('final_HIP.pkl','wb'))

In [22]:
pickle.dump(scaler1,open('scaler1.pkl','wb'))